E8.project_Movielens_Recommandation
===
---

넷플릭스에서는 사용자가 시청한 영화를 토대로 다른 영화들을 추천해줍니다. 이번 프로젝트로 CSR matrix를 사용하여 추천시스템 만들기를 도전해보겠습니다.


![netflix-ec82acec9aa9eab8b0_image-8](https://user-images.githubusercontent.com/70703320/107152448-46052b00-69ab-11eb-914f-7b8148b32956.png)

[이미지 출처](https://byline.network/2016/07/1-225/)

영화데이터에 대한 정보는 [이곳](https://biology-statistics-programming.tistory.com/47) 정리해주었습니다.



## 1. 데이터 준비와 전처리

### 필요한 라이브러리 불러오기

In [1]:
import os
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


### 데이터 불러와서 확인해보기

In [ ]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

### rating이 3점 이상인 것만 남겨주기

In [2]:
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


### rating컬럼의 이름을 count로 바꿔준 후 확인하기

In [3]:
ratings.rename(columns={'rating':'count'}, inplace=True)
ratings['count']

### 영화 제목을 보기 위해 다른 파일 불러오기

In [5]:
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding = "ISO-8859-1")
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


## 2. 분석 


### 유저 수

In [6]:
ratings['movie_id'].nunique()

3628

### 사용자 수

In [7]:
ratings['user_id'].nunique()

6039

### 가장 인기 많은 영화 30개 출력

In [51]:
# 앞선 두 데이터프레임을 합쳐서 출력해보기
mv_total = pd.merge(ratings, movies, how='left')
mv_total

mv_count = mv_total.groupby('title')['user_id'].count()
mv_count.sort_values(ascending=False).head(30)

title
Heidi Fleiss: Hollywood Madam (1995)              3212
Mortal Kombat (1995)                              2910
Young Poisoner's Handbook, The (1995)             2885
Two if by Sea (1996)                              2716
Pocahontas (1995)                                 2561
Mary Reilly (1996)                                2509
Boys of St. Vincent, The (1993)                   2498
Race the Sun (1996)                               2473
Copycat (1995)                                    2460
Star Maker, The (Uomo delle stelle, L') (1995)    2434
Muppet Treasure Island (1996)                     2413
It Takes Two (1995)                               2386
Guardian Angel (1994)                             2371
Dunston Checks In (1996)                          2314
Kids (1995)                                       2298
Assassins (1995)                                  2257
Father of the Bride Part II (1995)                2252
NeverEnding Story III, The (1994)                 2213
Don 

## 3. 내가 선호하는 영화를 5가지 골라서 rating에 추가

> 저의 픽은 American Beauty (1999),  Sixth Sense, The (1999), Men in Black (1997), Terminator, The (1984), Toy Story (1995) 입니다.

### movie_id 확인해보기

In [9]:
my_favorite = ['American Beauty (1999)' , 'Sixth Sense, The (1999)' ,'Men in Black (1997)' ,'Terminator, The (1984)' ,'Toy Story (1995)']

for movie in my_favorite:
    print(movies[movies['title'] == movie])
    print('--------------------------------------------------------------------')
    

      movie_id                   title         genre
2789      2858  American Beauty (1999)  Comedy|Drama
--------------------------------------------------------------------
      movie_id                    title     genre
2693      2762  Sixth Sense, The (1999)  Thriller
--------------------------------------------------------------------
      movie_id                title                           genre
1539      1580  Men in Black (1997)  Action|Adventure|Comedy|Sci-Fi
--------------------------------------------------------------------
      movie_id                   title                   genre
1220      1240  Terminator, The (1984)  Action|Sci-Fi|Thriller
--------------------------------------------------------------------
   movie_id             title                        genre
0         1  Toy Story (1995)  Animation|Children's|Comedy
--------------------------------------------------------------------


### 데이터 프레임에 추가해보기
> 여기서 저의 아이디는 hibiscircus, 별점은 5로 주었습니다.


In [10]:
my_mvid = [2858, 2762, 1580, 1240, 1]

# 960917이라는 아이디 추가
my_movielist = pd.DataFrame({'user_id': ['hibiscircus']*5, 'count':[5]*5, 'movie_id': [2858, 2762, 1580, 1240, 1]})

if not ratings.isin({'user_id':[960917]})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    ratings = ratings.append(my_movielist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,count,timestamp
1000203,6040,1090,3,956715518.0
1000205,6040,1094,5,956704887.0
1000206,6040,562,5,956704746.0
1000207,6040,1096,4,956715648.0
1000208,6040,1097,4,956715569.0
0,hibiscircus,2858,5,NaN
1,hibiscircus,2762,5,NaN
2,hibiscircus,1580,5,NaN
3,hibiscircus,1240,5,NaN
4,hibiscircus,1,5,NaN


### 사용하는 컬럼만 남겨주기 (timestamp 컬럼 제거)

In [11]:
using_cols = ['user_id', 'movie_id', 'count']
ratings = ratings[using_cols]
ratings

,user_id,movie_id,count
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
0,hibiscircus,2858,5
1,hibiscircus,2762,5
2,hibiscircus,1580,5
3,hibiscircus,1240,5


### user_id와 movie_id 고유한 값으로 만들기

> 여기서 아주 중요한 부분인 **원래의 값과 고유한 값을 이어주는 dictionary를 생성**

In [12]:
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data    
else:
    print('user_id column indexing Fail!!')

temp_movie_data = ratings['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('movie column indexing Fail!!')

ratings

## 4. CSR matrix 만들기

In [16]:
num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5. als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련

### 실행시키기 위해 필요한 부분

In [17]:
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

### Implicit AlternatingLeastSquares 모델의 선언

In [18]:
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

### als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose)

In [52]:
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

### model fitting

In [20]:
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

## 6. 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악

### 나의 아이디 hibiscircus 입력

In [33]:
hibiscircus = user_to_idx['hibiscircus']

### 내가 선호하는 영화 중 Toy Story (1995)를 입력

In [34]:
terminator = movie_to_idx[1]
terminator_vector = als_model.item_factors[terminator]
np.dot(hibiscircus_vector, terminator_vector)

0.45919308

### 그 외 영화 Jumanji (1995)를 입력

> 영화이름과 인덱스는 1. '영화 제목을 보기 위해 다른 파일 불러오기'에서 movie.dat을 불러왔을 때의 출력을 참조하였습니다.

In [35]:
jumanji = movie_to_idx[2]
jumanji_vector = als_model.item_factors[jumanji]
np.dot(hibiscircus_vector, jumanji_vector)

0.03269084

> 제가 선호하는 영화Toy Story (1995)에서의 값은 0.45919308이고, 그 외 영화Jumanji (1995)에서는 0.03269084의 값을 얻은 걸로 보아 제가 선호하는 영화에 좀 더 점수를 잘 주는 것으로 보입니다. 그렇다면 Toy Story (1995)유사한 영화들과의 유사도를 살펴보겠습니다.

###  Toy Story (1995)유사한 영화들과의 유사도

In [60]:
movie_id = movie_to_idx[1]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(40, 1.0),
 (50, 0.7749237),
 (33, 0.590679),
 (4, 0.5702657),
 (110, 0.56275433),
 (322, 0.5578007),
 (330, 0.4742893),
 (255, 0.4380188),
 (10, 0.4372744),
 (20, 0.3968088),
 (160, 0.37007928),
 (478, 0.36247167),
 (32, 0.3557714),
 (34, 0.35285798),
 (126, 0.3521961)]

> 어떤 영화인지 살펴봐주는 함수를 만들어보겠습니다.

## 7.내가 좋아하는 영화와 비슷한 영화를 추천받기

In [42]:
def get_similar_movie(favorite_movie):
    movie_id = movie_to_idx[favorite_movie]
    similar_movie = als_model.similar_items(movie_id, N=15)
    idx_to_movie = {v:k for k,v in movie_to_idx.items()}
    recommand_list = [idx_to_movie[i[0]] for i in similar_movie]
    for movie in recommand_list:
        print(movies[movies['movie_id'] == movie], '--------------------------------------------------------------------', sep='\n')

get_similar_movie(1)    

   movie_id             title                        genre
0         1  Toy Story (1995)  Animation|Children's|Comedy
--------------------------------------------------------------------
      movie_id               title                        genre
3045      3114  Toy Story 2 (1999)  Animation|Children's|Comedy
--------------------------------------------------------------------
     movie_id           title                                genre
584       588  Aladdin (1992)  Animation|Children's|Comedy|Musical
--------------------------------------------------------------------
      movie_id                 title                        genre
2286      2355  Bug's Life, A (1998)  Animation|Children's|Comedy
--------------------------------------------------------------------
      movie_id                 title           genre
1245      1265  Groundhog Day (1993)  Comedy|Romance
--------------------------------------------------------------------
    movie_id        title            

## 8. 내가 가장 좋아할 만한 영화들을 추천받기

In [50]:
def recommand_movie(user_id):
    user = user_to_idx[user_id]
    movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
    recommand_list = [idx_to_movie[i[0]] for i in movie_recommended]
    for movie in recommand_list:
        print(movies[movies['movie_id'] == movie], '--------------------------------------------------------------------', sep='\n')

recommand_movie(hibiscircus)

     movie_id                    title         genre
898       910  Some Like It Hot (1959)  Comedy|Crime
--------------------------------------------------------------------
      movie_id               title           genre
2992      3061  Holiday Inn (1942)  Comedy|Musical
--------------------------------------------------------------------
      movie_id                   title    genre
3606      3675  White Christmas (1954)  Musical
--------------------------------------------------------------------
    movie_id        title                    genre
33        34  Babe (1995)  Children's|Comedy|Drama
--------------------------------------------------------------------
      movie_id                       title    genre
1022      1035  Sound of Music, The (1965)  Musical
--------------------------------------------------------------------
     movie_id                                   title  \
590       594  Snow White and the Seven Dwarfs (1937)   

                            ge

## 9. 내가 좋아할만한 영화에 대해 기여해준 영화와 기여도 확인

> 저에게 가장 우선순위로 추천해준 Some Like It Hot (1959) 영화에 대해 어떤 영화가 얼마나 기여를 해주었는지 확인해보겠습니다.

### 기여도

In [63]:
some_like_it_hot = movie_to_idx[910]
explain = als_model.explain(user, csr_data, itemid=some_like_it_hot)
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[(2858, 0.028630675715682657),
 (1, 0.018044241733943013),
 (2762, 0.014292158838007989),
 (1240, 0.004356279763579437),
 (1580, -0.006033012421905186)]

### 기여한 영화

In [58]:
contribution = [idx_to_movie[i[0]] for i in explain[1]]
for movie in contribution:
    print(movies[movies['movie_id'] == movie], '--------------------------------------------------------------------', sep='\n')


      movie_id                   title         genre
2789      2858  American Beauty (1999)  Comedy|Drama
--------------------------------------------------------------------
   movie_id             title                        genre
0         1  Toy Story (1995)  Animation|Children's|Comedy
--------------------------------------------------------------------
      movie_id                    title     genre
2693      2762  Sixth Sense, The (1999)  Thriller
--------------------------------------------------------------------
      movie_id                   title                   genre
1220      1240  Terminator, The (1984)  Action|Sci-Fi|Thriller
--------------------------------------------------------------------
      movie_id                title                           genre
1539      1580  Men in Black (1997)  Action|Adventure|Comedy|Sci-Fi
--------------------------------------------------------------------


> 역시나 제가 리뷰한 영화들이 나왔습니다. 분석을 해보자면 Some Like It Hot (1959)영화의 장르는 Comedy|Crime입니다. 그런데 제가 좋아하는 영화 5개의 종류는 comedy아니면 Thriller입니다. 이를 잘 적절하게 조율해서 추천을 해준 듯 합니다. 그렇지만 저 영화는 안 볼겁니다.

## 회고

영화 id와 영화이름을 매칭하는 것의 어려움
- 처음에는 rating.dat과 movies.dat을 합쳐서 분석하려고 했습니다. 하지만, 필요없는 컬럼이 생겨서 난잡하기만 했고, 결국에는 영화를 인기순으로 출력하며 count를 같이 출력할때만 만들었습니다. 그 이외에는 필요할 때마다 호출하는 식으로 하기로 하였습니다.

CSR matrix의 생성의 어려움
- CSR matrix의 생성을 위해 csr_matrix를 사용하였을 때 shape를 지정해주는 경우 에러가 발생하였습니다. 이로 인해 원인을 찾다가 shape를 지정해주지 않아도 자동으로 설정해준다는 것을 알게되어 지우니 해결되었습니다.


## 루브릭

사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.
- 4.CSR matrix 만들기에서 확인이 가능합니다. 6040x3628의 크기로 만들어졌습니다.

사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.
- 6. 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악에서 확인이 가능합니다. 제가 선호하는 영화의 내적수치는 0.45919308, 그 외 영화의 내적수치는 0.03269084가 나온 것으로 보아 의미가 있다고 볼 수 있습니다. (원래 rating이라는 변수는 명시적 평가이지만 이번 프로젝트를 위해 암묵적 평가로 보았다. 따라서, 내적수치는 의미가 있을 수 밖에 없다.)

MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.
- 6. 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악과 9. 내가 좋아할만한 영화에 대해 기여해준 영화와 기여도 확인 확인이 가능합니다.